<a href="https://colab.research.google.com/github/tasinfrancesco/Practical_ML_PSL/blob/main/ML_submission_Tasin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
df = pd.read_csv("train.csv")


In [ ]:
# print(df.describe())
print(df.head(5))
# dt = pd.read_csv("test.csv")
# print(dt[["UTC" not in cd for cd in dt["creation_date"]]])
type(df["creation_date"][0])
print(strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z"))

print(strptime(df["creation_date"][1], "%Y-%m-%d %H:%M:%S.%f %Z"))
print(df["creation_date"][0])
print(df["creation_date"][1])

def timeinfo(timestr):
  time_struct = strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z")

  #is it a weekend
  if time_struct.tm_wday >= 5:
    weekend = 1
  else:
    weekend = 0

  #is it a vacation



Ideas for feature engineering:

  - local comment frequency (estimated from change in post_id over time window)
  - is it a vacation
  - is it a weekend

post

  - post length
  - positivity of content?
  - politeness?

BERT:
  - encode in some way politeness? (there's a stanford thingy)
  - CodeBERT vs BERT?
  - hide vectors of code/cut it out for text processing?
  - clarity as:
    - readability
    - sentence/word length
    - coding depth (nesting)
    - number of questions
    - presence of errors
    - number of topics mentioned ("I tried...")

User ID?

  - is it better to predict for a given user (average) or for single posts?

text vectorisation:

analysis:

  - logistic regression (not great given collinearity)
  - random trees?

predict score, then guess target (not target directly)


In [3]:
# !pip install sentencepiece
# !pip install protobuf==3.20.0
from collections import defaultdict, Counter
import json
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")
model = AutoModelForTokenClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
outputs_1_to_10 = []
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]
for t in df["text"][0:10]:
  tok = tokenizer.tokenize(t)
  ids = tokenizer.convert_tokens_to_ids(tok)
  ids_special_tokens = cls + ids + sep

  decoded_str = tokenizer.decode(ids_special_tokens)

  print("start:               ", t)
  print("tokenize:            ", tok)
  print("convert_tokens_to_ids", ids)
  print("add special tokens:  ", ids_special_tokens)
  print("================")
  print("decode:              ", decoded_str)
  print("\n\n")




start:                Sincerely appreciate your taking a look. I never have to query across partitions. Disk space is not an issue. The volume of data definitely justifies the partitioning. Separate tables and or databases are not the option. Still my question remains unanswered :-( Many thanks in advance.
tokenize:             ['Sincerely', 'appreciate', 'your', 'taking', 'a', 'look', '.', 'I', 'never', 'have', 'to', 'query', 'across', 'partitions', '.', 'Disk', 'space', 'is', 'not', 'an', 'issue', '.', 'The', 'volume', 'of', 'data', 'definitely', 'justifies', 'the', 'partitioning', '.', 'Separate', 'tables', 'and', 'or', 'databases', 'are', 'not', 'the', 'option', '.', 'Still', 'my', 'question', 'remains', 'unanswered', ':', '-', '(', 'Many', 'thanks', 'in', 'advance', '.']
convert_tokens_to_ids [52297, 5429, 1881, 4670, 69, 2076, 18, 45, 3071, 1851, 1790, 2400, 4036, 12207, 18, 17326, 3257, 1804, 1859, 1794, 2456, 18, 1911, 7817, 1815, 1943, 5480, 57754, 1783, 14744, 18, 21624, 3133

In [13]:
t0 = df["text"][0]
tokenizer.encode(t0, return_tensors = "pt")
model.

TokenClassifierOutput(loss=None, logits=tensor([[[-0.6075,  1.1530, -0.6116,  ...,  0.3006, -0.6447,  0.1319],
         [-0.0289,  1.5941, -0.6061,  ...,  0.8019, -0.4946, -0.2197],
         [-0.5585,  1.2193,  0.0123,  ...,  0.0702, -0.0290,  0.7848],
         ...,
         [-0.3575,  0.2732, -0.6512,  ...,  0.9173, -0.8593, -0.1063],
         [-0.6911,  1.1412, -0.5137,  ...,  0.7186, -0.4854,  0.0965],
         [-1.2088,  0.3238, -0.3494,  ...,  0.8452, -0.2732,  0.0849]]],
       grad_fn=<ViewBackward0>), hidden_states=(tensor([[[ 5.2508e-02,  1.0643e+00, -1.6286e+00,  ..., -2.1369e-01,
          -1.1551e+00,  8.0956e-01],
         [ 1.4543e-01,  6.2975e-01, -1.4977e+00,  ...,  6.4980e-01,
          -8.5041e-01,  7.1804e-01],
         [-8.8615e-01,  1.7793e+00, -4.2127e-01,  ...,  4.9867e-01,
          -1.2349e+00, -8.6165e-01],
         ...,
         [ 1.9428e+00, -1.0674e+00, -1.7318e-01,  ...,  3.8954e-01,
           1.3849e-01,  1.3046e+00],
         [ 9.9077e-01,  1.1217e+00, 